In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

In [ ]:
top_authors = df.groupby("author").count().sort_values("title", ascending=False).head(20)[["title"]]

In [ ]:
min_articles = min(top_authors["title"])

In [ ]:
adf = pd.concat([df[df["author"] == author].sample(min_articles, random_state=42)
                     for author in top_authors.index.values])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=2)
tfidf_vectors = tfidf_vectorizer.fit_transform(adf["nav"])
tfidf_vectors

In [ ]:
import numpy as np

def train_and_predict(clf, X, y):
    clf.fit(X, y)
    predicted = clf.predict(X)
    print(np.unique(y == predicted, return_counts=True))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
train_and_predict(MultinomialNB(), tfidf_vectors, adf["author"].values)

In [ ]:
from sklearn.linear_model import LogisticRegression
train_and_predict(LogisticRegression(), tfidf_vectors, adf["author"].values)

In [ ]:
from sklearn.linear_model import SGDClassifier
train_and_predict(SGDClassifier(loss="hinge"), tfidf_vectors, adf["author"].values)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
train_and_predict(DecisionTreeClassifier(), tfidf_vectors, adf["author"].values)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
train_and_predict(RandomForestClassifier(), tfidf_vectors, adf["author"].values)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
train_and_predict(GradientBoostingClassifier(), tfidf_vectors, adf["author"].values)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
train_and_predict(KNeighborsClassifier(), tfidf_vectors, adf["author"].values)